In [1]:
from src import *

In [2]:
# Set path to file
amr_path = '../data/AMR24_Dx_Capacity_assessment_in_counties_-_all_versions_-_labels_-_2023-10-24-09-26-34.xlsx'

# Load data
amr_df = pd.read_excel(amr_path)

# Inspect
amr_df.head()

,start,end,"The Ministry of Health, in collaboration with JKUAT, FIND, the global alliance for diagnostics, and GARDP is carrying out an activity to assess AMR diagnostic capacity, antibiotic use and existing antimicrobial stewardship practices in preparation for the introduction of cefiderocol (and other antibiotics) and new low blood culture and molecular Point of care testing (POCT) platforms in Kenya. The activity is taking place at selected levels 4,5 and 6 of health facilities in Isiolo, Laikipia, Vihiga, Nyeri, Nairobi, Kirinyaga, Kericho, Kajiado, Nandi and Kilifi counties. \nPlease complete this questionnaire for the facilities that you are reporting on, and where indicated, provide additional details on your selected responses. \n\nFindings will be published by MoH, Isiolo, Laikipia, Vihiga, Nyeri, Nairobi, Kirinyaga, Kericho, Kajiado, Nandi and Kilifi Counties, as well as JKUAT, FIND and GARDP.",County,Sub county,Ward,Name of health facility,Facility code,Assessor name,GPS coordinates,...,_notes,_status,_submitted_by,__version__,_tags,_index,Unnamed: 375,Unnamed: 376,Unnamed: 377,Unnamed: 378
0,2023-09-19 09:38:40.129,2023-09-19 12:10:07.529,NaN,Isiolo,Isiolo,Wabera,Isiolo County and Referral Hospital,12094,GEORGE MUCHIRI NJAU,0.3647273 37.5896959 1081.5 25.136,...,NaN,submitted_via_web,amr_abx,vDeNhdSRZ7scoW8UrHD6hm,NaN,1,NaN,NaN,NaN,NaN
1,2023-09-20 09:14:18.598,2023-09-20 11:26:12.117,NaN,Isiolo,Isiolo,Bulla Pesa,Anka Hospital Isiolo,29530,GEORGE MUCHIRI NJAU,0.3498341 37.5820279 1093.7000732421875 15.5,...,NaN,submitted_via_web,amr_abx,v7dgbgEbVY8RrodPdYpYiU,NaN,2,1.0,0.0,1.0,0.0
2,2023-09-20 10:28:13.467,2023-09-20 15:14:25.455,NaN,Nairobi,Ruaraka,Korogocho,Mama Margaret Uhuru Hospital,28885,Peter Kinyanjui,-1.2513671 36.8864047 1584.300048828125 20.0,...,NaN,submitted_via_web,amr_abx,v7dgbgEbVY8RrodPdYpYiU,NaN,3,0.0,0.0,0.0,1.0
3,2023-09-20 10:53:46.163,2023-09-20 16:28:45.251,NaN,Nandi,Emgwen,Kapsabet,Kapsabet County Referral Hospital,14749,Mmboyi Onesmus,0.2021363 35.10287 1985.300048828125 5.0,...,NaN,submitted_via_web,amr_abx,v7dgbgEbVY8RrodPdYpYiU,NaN,4,NaN,NaN,NaN,NaN
4,2023-09-21 08:15:34.204,2023-09-21 09:40:45.038,NaN,Isiolo,Isiolo,Bulla Pesa,MaterCare Maternity Hospital,19912,GEORGE MUCHIRI NJAU,0.3274352 37.5638683 1190.0 4.957,...,NaN,submitted_via_web,amr_abx,v7dgbgEbVY8RrodPdYpYiU,NaN,5,NaN,NaN,NaN,NaN


In [3]:
def get_prefixes(df: pd.DataFrame) -> tuple:
    prefixes = [col.split(' ', 1)[0] + ' ' for col in df.columns if col.count('/') > 0]
    prefix_counts = {prefix: prefixes.count(prefix) for prefix in prefixes}
    prefixes = [prefix for prefix in prefixes if prefix_counts[prefix] > 1]
    n_prefixes = len(prefixes)
    unique_prefixes = list(set(prefixes))
    n_unique_prefixes = len(unique_prefixes)

    return prefixes, unique_prefixes, n_prefixes, n_unique_prefixes


def assign_cols(df: pd.DataFrame, prefixes: list) -> tuple:
    initial_cols = []

    for prefix in prefixes:
        for col in df.columns:
            if col.startswith(prefix):
                initial_cols.append(col)
                break

    encoded_cols = []

    for col in df.columns:
        for prefix in prefixes:
            if col.startswith(prefix):
                encoded_cols.append(col)

    encoded_cols = [col for col in encoded_cols if col not in initial_cols]

    return initial_cols, encoded_cols


def create_col_dict(initial_cols: list, encoded_cols: list) -> dict:
    col_dict = {}

    for col in encoded_cols:
        prefix = col.split(' ', 1)[0] + ' '
        for col_ in initial_cols:
            if col_.startswith(prefix):
                col_dict[col] = col_
                break

    return col_dict


def create_values_replace_map(col_dict: dict) -> dict:
    val_dict = {}

    for k, v in col_dict.items():
        val_dict[k] = k.replace(v, '')

    for k, v in val_dict.items():
        if v.startswith('/'):
            val_dict[k] = v[1:]

    return val_dict


def create_names_replace_map(initial_cols: list, unique_prefixes: list) -> dict:
    names_dict = {}

    old = [prefix + 'value' for prefix in unique_prefixes]

    for i in old:
        for col in initial_cols:
            if i.split(' ', 1)[0] in col:
                names_dict[i] = col
                break

    return names_dict


def replace_values(df: pd.DataFrame, initial_cols: list, encoded_cols: list, val_dict: dict):
    for col in encoded_cols:
        df[col] = df[col].replace({1:val_dict[col], 0:np.nan})
    
    # TODO remove operation (can be performed independently)
    df = df.drop(initial_cols, axis=1)
    
    return df


def melt_cols(df: pd.DataFrame, prefixes: list):
    if not isinstance(prefixes, list):
        prefixes = list(prefixes)

    for prefix in prefixes:
        cols = [col for col in df.columns if col.startswith(prefix)]
        df = df.melt(id_vars=[col for col in df.columns if col not in cols],
                     value_vars=cols,
                     var_name=prefix,
                     value_name=prefix + 'value')
        
    df = df.drop(prefixes, axis=1).drop_duplicates()

    return df


# def clean_up_df()

In [4]:
prefixes, unique_prefixes, n_prefixes, n_unique_prefixes = get_prefixes(amr_df)
initial_cols, encoded_cols = assign_cols(amr_df, unique_prefixes)
col_dict = create_col_dict(initial_cols, encoded_cols)
val_dict = create_values_replace_map(col_dict)
names_map = create_names_replace_map(initial_cols, unique_prefixes)
amr = replace_values(amr_df, initial_cols, encoded_cols, val_dict)
amr = melt_cols(amr, sorted(unique_prefixes, reverse=True)[:5])
# amr.rename(columns=names_map, inplace=True)
amr.iloc[:,-5:]

,i2 value,h1. value,f1. value,e2. value,d9. value
0,NaN,NaN,Out-of-pocket,NaN,NaN
1,NaN,NaN,Out-of-pocket,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,SLIPTA program,Out-of-pocket,NaN,NaN
4,NaN,NaN,Out-of-pocket,NaN,NaN
...,...,...,...,...,...
121117,other,SLMTA program,Free,Lab Personnel,NaN
121145,NaN,NaN,Free,Lab Personnel,NaN
121313,other,NaN,Free,Lab Personnel,NaN
121537,NaN,SLIPTA program enrollment ongoing,Free,Lab Personnel,NaN


In [5]:
amr = melt_cols(amr, sorted(unique_prefixes, reverse=True)[5:10])
# amr.rename(columns=names_map, inplace=True)
amr.iloc[:,-5:]

,d80 value,d8. value,d71 value,d42 value,d41 value
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Disk diffusion
4,NaN,NaN,NaN,NaN,Disk diffusion
...,...,...,...,...,...
427465,NaN,Automated,NaN,NaN,Agar dilution
427475,NaN,Automated,NaN,NaN,Agar dilution
427477,NaN,Automated,NaN,NaN,Agar dilution
427478,NaN,Automated,NaN,NaN,Agar dilution


In [6]:
amr = melt_cols(amr, sorted(unique_prefixes, reverse=True)[10:15])
# amr.rename(columns=names_map, inplace=True)
amr.iloc[:,-5:]

MemoryError: Unable to allocate 417. MiB for an array with shape (9, 6076800) and data type float64

In [ ]:
amr = melt_cols(amr, sorted(unique_prefixes, reverse=True)[15:20])
# amr.rename(columns=names_map, inplace=True)
amr.iloc[:,-5:]

In [ ]:
amr = melt_cols(amr, sorted(unique_prefixes, reverse=True)[20:23])
# amr.rename(columns=names_map, inplace=True)
amr.iloc[:,-5:]

In [ ]:
# Get the unique values for each of the last 5 columns
for col in amr.iloc[:,-23:]:
    print(col)
    print('-' * len(col))
    print(amr[col].value_counts())

In [ ]:
amr.iloc[:,-5:] = amr.iloc[:,-5:].replace({'None.1': 'None',
                                           'other': 'Other',
                                           }).drop_duplicates()

amr.iloc[:,-5:]

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(amr, x='Name of health facility', hue='h1. value', legend=True, palette=palette, orient='h')
plt.xticks(rotation=90)
plt.show()

In [ ]:
print(sorted(unique_prefixes))

In [ ]:
print(len(prefixes), len(unique_prefixes))